### **<Telco Customer Churn – Modeling>**

본 노트북은 EDA에서 도출된 데이터셋을 기반으로  
고객 이탈 여부(Churn)를 예측하는 머신러닝 모델을 구축하는 단계입니다.

EDA에서는 비즈니스 인사이트 도출에 집중하였으며,  
본 단계에서는 동일한 전처리 기준을 유지한 데이터를 활용하여  
예측 성능 관점에서 모델을 평가합니다.  

#### **1. 🎯 문제 정의** 
**Problem Definition**

- 목표: 고객의 이탈 여부(Churn)를 예측하는 이진 분류 문제
- 활용 데이터: EDA에서 전처리 완료된 고객 데이터
- 기대 효과:
  - 이탈 가능성이 높은 고객 사전 식별
  - 타겟 마케팅 및 유지 전략 수립

#### **2. 데이터 로드 및 불필요 컬럼 제거**

In [1]:
import pandas as pd
df = pd.read_csv("../data/cleaned_churn.csv")
df = df.drop(columns=['customerID', 'tenure_group'])
df.shape

(7032, 20)

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7032 entries, 0 to 7031
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7032 non-null   object 
 1   SeniorCitizen     7032 non-null   int64  
 2   Partner           7032 non-null   object 
 3   Dependents        7032 non-null   object 
 4   tenure            7032 non-null   int64  
 5   PhoneService      7032 non-null   object 
 6   MultipleLines     7032 non-null   object 
 7   InternetService   7032 non-null   object 
 8   OnlineSecurity    7032 non-null   object 
 9   OnlineBackup      7032 non-null   object 
 10  DeviceProtection  7032 non-null   object 
 11  TechSupport       7032 non-null   object 
 12  StreamingTV       7032 non-null   object 
 13  StreamingMovies   7032 non-null   object 
 14  Contract          7032 non-null   object 
 15  PaperlessBilling  7032 non-null   object 
 16  PaymentMethod     7032 non-null   object 


In [3]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


#### **3. 🔧 전처리**
**Data Preprocessing**

EDA에서는 해석 중심으로 일부 변수를 제외하였으나,  
모델링 단계에서는 예측 성능을 고려하여 전체 변수를 활용합니다.

#### **3-1. 타겟 변수 인코딩**

In [4]:
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

#### **3-2. Feature / Target 분리**

In [5]:
X = df.drop(columns=['Churn'])
y = df['Churn']

In [6]:
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

#### **3-3. 범주형 변수 인코딩**

In [7]:
cat_cols = X.select_dtypes(include='object').columns
X_encoded = pd.get_dummies(
    X, 
    columns=cat_cols, 
    drop_first=True
)
X_encoded.shape

(7032, 30)

#### **3-4. Train / Test split**

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

- 모델 학습 및 평가 과정에서 데이터 누수를 방지하기 위해  
  train / test 데이터를 분리한 후 모든 모델 학습 및 threshold 튜닝은  
  train 데이터 기반으로 수행

#### **4. Logistic Regression(baseline) 학습**

In [9]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

/home/mars/churn_project/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

#### **4-1. Logistic Regression 예측**

In [10]:
y_pred = model.predict(X_test)

#### **4-2. Logistic Regression 성능 평가 + ROC-AUC**

In [11]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix\n", confusion_matrix(y_test, y_pred))

# ROC-AUC
y_proba_log = model.predict_proba(X_test)[:, 1]
roc_auc_log = roc_auc_score(y_test, y_proba_log)
print("Logistic Regression ROC-AUC:", round(roc_auc_log, 3))

Accuracy: 0.8031272210376688

Classification Report
               precision    recall  f1-score   support

           0       0.85      0.89      0.87      1033
           1       0.65      0.57      0.61       374

    accuracy                           0.80      1407
   macro avg       0.75      0.73      0.74      1407
weighted avg       0.80      0.80      0.80      1407


Confusion Matrix
 [[917 116]
 [161 213]]
Logistic Regression ROC-AUC: 0.836


#### **4-3. 변수 중요도 (계수 해석)**

In [17]:
coef_df = pd.DataFrame({
    'feature': X_encoded.columns,
    'coef': model.coef_[0]
})

coef_df = coef_df.sort_values(by='coef', ascending=False)

coef_df.head(10)      # 이탈 증가 요인

,feature,coef
10,InternetService_Fiber optic,0.685620
28,PaymentMethod_Electronic check,0.388557
26,PaperlessBilling_Yes,0.296361
9,MultipleLines_Yes,0.278560
8,MultipleLines_No phone service,0.253005
21,StreamingTV_Yes,0.203571
0,SeniorCitizen,0.194216
23,StreamingMovies_Yes,0.175202
29,PaymentMethod_Mailed check,0.077442
27,PaymentMethod_Credit card (automatic),0.044777


In [ ]:
coef_df.tail(10)      # 이탈 감소 요인

,feature,coef
18,TechSupport_No internet service,-0.105487
16,DeviceProtection_No internet service,-0.105487
20,StreamingTV_No internet service,-0.105487
15,OnlineBackup_Yes,-0.213772
6,Dependents_Yes,-0.229065
19,TechSupport_Yes,-0.414394
7,PhoneService_Yes,-0.439617
13,OnlineSecurity_Yes,-0.454926
24,Contract_One year,-0.752908
25,Contract_Two year,-1.380015


#### **4-4. 로지스틱 결과 해석**
- 모델 정확도는 약 80% 수준으로 안정적인 성능을 보였으나,  
이탈 고객(Churn=1)에 대한 recall은 약 57%로 상대적으로 낮게 나타남    

- 이는 실제 이탈 고객 중 약 43%를 모델이 탐지하지 못하는 구조로,  
이탈 방지 관점에서는 중요한 고객을 놓칠 수 있는 한계를 의미  

- 따라서 단순 정확도보다 이탈 고객 탐지 성능(recall)을 개선하는 방향의  
모델 튜닝 필요  

- 특히 고객 이탈 문제에서는 False Negative(이탈 고객을 놓치는 경우)가 비지니스 손실로 직결되므로,  
recall을 우선적으로 개선하는 전략이 중요

클래스 불균형 문제를 고려하여  
class_weight 조정 또는 threshold 튜닝을 통해 이탈 고객 탐지 성능을 개선 가능

#### **5. Random Forest(baseline) 학습**

In [12]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=100,
    random_state=42
)

rf.fit(X_train, y_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric

#### **5-1. Random Forest 예측**

In [13]:
y_pred_rf = rf.predict(X_test)

#### **5-2. Random Forest 성능 평가 + ROC-AUC**

In [14]:
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("\nClassification Report\n", classification_report(y_test, y_pred_rf))
print("\nConfusion Matrix\n", confusion_matrix(y_test, y_pred_rf))

# ROC-AUC
y_proba_rf = rf.predict_proba(X_test)[:, 1]
roc_auc_rf = roc_auc_score(y_test, y_proba_rf)
print("Random Forest ROC-AUC:", round(roc_auc_rf, 3))

Accuracy: 0.7896233120113717

Classification Report
               precision    recall  f1-score   support

           0       0.84      0.89      0.86      1033
           1       0.63      0.52      0.57       374

    accuracy                           0.79      1407
   macro avg       0.73      0.70      0.71      1407
weighted avg       0.78      0.79      0.78      1407


Confusion Matrix
 [[917 116]
 [180 194]]
Random Forest ROC-AUC: 0.816


#### **5-3. 변수 중요도**

In [22]:
feature_importance = pd.DataFrame({
    'feature': X_encoded.columns,
    'importance': rf.feature_importances_
})

feature_importance = feature_importance.sort_values(
    by='importance',
    ascending=False
)

feature_importance.head(10)

,feature,importance
3,TotalCharges,0.191435
1,tenure,0.171020
2,MonthlyCharges,0.168400
10,InternetService_Fiber optic,0.039481
28,PaymentMethod_Electronic check,0.037416
25,Contract_Two year,0.030529
4,gender_Male,0.029332
13,OnlineSecurity_Yes,0.028157
26,PaperlessBilling_Yes,0.025536
19,TechSupport_Yes,0.024114


#### **5-4. 랜덤포레스트 결과 해석**
- RandomForest 모델은 정확도 약 79%로 Logistic Regression 대비 소폭 낮은 성능을 보였으며,  
특히 이탈 고객(Churn=1)에 대한 recall이 더 낮게 나타남  

- 이는 이탈 고객 탐지 관점에서는 Logistic Regression이 더 적합한 모델임을 의미  

- 그러나 RandomForest는 변수 중요도(feature importance)를 통해  
예측에 영향을 미치는 주요 변수들을 직관적으로 파악할 수 있다는 장점이 있음  

- 변수 중요도 분석 결과, Contract 유형, tenure, MonthlyCharges 등의 변수가  
이탈 예측에 중요한 역할을 하는 것으로 나타났으며,  
이는 EDA에서 도출한 인사이트와 일관된 결과  

따라서 RandomForest는 최종 모델로 선택되지는 않았지만,  
변수 영향력 파악 및 EDA 인사이트 검증을 위한 보조 모델로 활용  

#### **6. 모델 개선**
#### **6-1. Logistic + class_weight**

In [15]:
model_bal = LogisticRegression(
    max_iter=1000,
    class_weight='balanced'
)
model_bal.fit(X_train, y_train)

/home/mars/churn_project/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",'balanced'
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :ter

#### **6-2. threshold 조정**

In [17]:
y_proba = model_bal.predict_proba(X_test)[:, 1]

thresholds = [0.3, 0.4, 0.5]

for t in thresholds:
    y_pred_t = (y_proba >= t).astype(int)
    print(f"\nThreshold: {t}")
    print(classification_report(y_test, y_pred_t))

# 최종 선택 threshold
y_pred_03 = (y_proba >= 0.3).astype(int)


Threshold: 0.3
              precision    recall  f1-score   support

           0       0.95      0.52      0.68      1033
           1       0.41      0.93      0.57       374

    accuracy                           0.63      1407
   macro avg       0.68      0.72      0.62      1407
weighted avg       0.81      0.63      0.65      1407


Threshold: 0.4
              precision    recall  f1-score   support

           0       0.93      0.61      0.74      1033
           1       0.45      0.87      0.59       374

    accuracy                           0.68      1407
   macro avg       0.69      0.74      0.67      1407
weighted avg       0.80      0.68      0.70      1407


Threshold: 0.5
              precision    recall  f1-score   support

           0       0.91      0.70      0.79      1033
           1       0.49      0.80      0.61       374

    accuracy                           0.73      1407
   macro avg       0.70      0.75      0.70      1407
weighted avg       0.80   

#### **6-3. 성능 평가 + ROC-AUC**

In [18]:
print("Accuracy:", accuracy_score(y_test, y_pred_03))
print("\nClassification Report\n", classification_report(y_test, y_pred_03))
print("\nConfusion Matrix\n", confusion_matrix(y_test, y_pred_03))

# ROC-AUC (threshold-independent)
roc_auc_bal = roc_auc_score(y_test, y_proba)
print("Logistic Regression (balanced) ROC-AUC:", round(roc_auc_bal, 3))

Accuracy: 0.6304193319118693

Classification Report
               precision    recall  f1-score   support

           0       0.95      0.52      0.68      1033
           1       0.41      0.93      0.57       374

    accuracy                           0.63      1407
   macro avg       0.68      0.72      0.62      1407
weighted avg       0.81      0.63      0.65      1407


Confusion Matrix
 [[541 492]
 [ 28 346]]
Logistic Regression (balanced) ROC-AUC: 0.836


#### **6-4. 성능 비교**
| 모델                  | Accuracy | Recall | Precision | ROC-AUC |
| ------------------- | -------- | ------ | --------- | ------- |
| Logistic (baseline) | 0.80     | 0.57   | 0.65      | 0.83    |
| RandomForest        | 0.79     | 0.52   | 0.63      | 0.81    |
| Logistic (tuned)    | 0.63     | 0.93   | 0.41      | 0.83    |

#### **6-5. Threshold 선택 기준**

- threshold(0.3, 0.4, 0.5)를 비교한 결과,  
  threshold가 낮아질수록 이탈 고객 recall이 크게 증가하는 경향을 확인

- 특히 0.3에서 recall이 가장 높은 수준을 보였으며,  
  고객 이탈을 놓치지 않는 것이 중요한 비즈니스 목적에 부합한다고 판단

- 따라서 recall 성능과 비즈니스 목적을 고려하여  
  최종 threshold를 0.3으로 설정

- 다만 threshold를 낮추면서 precision은 감소하였으며,  
  이는 일부 비이탈 고객을 이탈 고객으로 잘못 분류하는 trade-off가 존재함

#### **7. 최종 모델 선정**
- 본 프로젝트에서는 Logistic Regression 모델을 최종 모델로 선정
- threshold를 0.3으로 조정하여 이탈 고객 탐지 기능(recall)을 개선  
    → 약 90% 이상의 이탈 고객을 탐지할 수 있는 수준으로 향상
- 최종적으로,
    - Accuracy보다 Recall(이탈 감지)을 우선 기준으로 설정
    - 비즈니스 관점에서 "이탈 고객을 놓치지 않는 것"을 더 중요하게 판단

#### **7-1. 이탈 확률 기반 고객 분류**

In [28]:
result_df = X_test.copy()
result_df['Churn_Prob'] = y_proba
result_df['Churn_Pred'] = y_pred_03

#### **7-2. 위험군 정의**

In [29]:
def risk_group(p):
    if p >= 0.7:
        return "High Risk"
    elif p >= 0.4:
        return "Medium Risk"
    else:
        return "Low Risk"

result_df['Risk_Group'] = result_df['Churn_Prob'].apply(risk_group)

#### **7-3. 분포 확인**

In [30]:
result_df['Risk_Group'].value_counts()

Risk_Group
Low Risk       683
High Risk      376
Medium Risk    348
Name: count, dtype: int64

#### **7-4. 이탈 위험군 세그먼트 정의**

- 모델이 예측한 이탈 확률을 기준으로 고객을 위험군으로 분류
  - High Risk (0.7 이상)
  - Medium Risk (0.4 ~ 0.7)
  - Low Risk (0.4 미만)

- 이를 통해 단순 예측을 넘어,
  실제 운영에서 활용 가능한 고객 세그먼트로 확장

- 특히 High Risk 고객군은
  우선적인 유지 전략(할인, 혜택 제공 등)의 대상이 될 수 있음